# Step 0: Imports + Utilities

In [1]:
from copy import deepcopy
from itertools import product, groupby
import os

In [2]:
def neg(x):
    if '-' in x:
        return x.replace('-', '')
    else:
        return '-' + x

In [3]:
# This is intentionally hard-coded, because Notebooks does not have the concept of '__file__'
# TODO: change this in PLRes.py
BASE_DIR   = '..'
INPUT_DIR  = BASE_DIR + '/input'
OUTPUT_DIR = BASE_DIR + '/output'

input_file  = lambda filename: INPUT_DIR + '/' + filename
output_file = lambda filename: OUTPUT_DIR + '/' + filename

# Step 1: Read KB and alpha

In [4]:
def parse(file_object):
    # Init
    lines = []
    alpha = []
    kb = []
    alpha_count = 0
    kb_count = 0

    # Read file
    lines = file_object.read().splitlines()
    
    # Read alpha
    alpha_count = int(lines.pop(0))
    while alpha_count > 0:
        l = lines.pop(0)
        l = [[c.strip() for c in l.split(' OR ')]]
        alpha.extend(l)
        alpha_count -= 1
    
    # Read KB
    kb_count = int(lines.pop(0))
    while kb_count > 0:
        l = lines.pop(0)
        l = [[c.strip() for c in l.split(' OR ')]]
        kb.extend(l)
        kb_count -= 1
    
    return alpha, kb

In [5]:
def neg_alpha(alpha):
    """Alpha negation. "It's almost CNF, but not quite."""
    r = []
    
    # Negate all clauses
    for each in alpha:
        r.append([neg(c) for c in each])

    # Perform "CNF"
    r = list(product(*r))

    # Eliminate tautologies
    r2 = []
    for i, each in enumerate(r):
        if all(neg(each_c) not in each for each_c in each):
            r2.append(each)
    r = r2

    # Eliminate duplicate literals
    r = [
        sorted(set(each), key=lambda c: c.replace('-', '') if '-' in c else c)
        for each in r
    ]

    # Elimiate duplicate clauses
    r = list(each for each, _ in groupby(sorted(r)))
    
    return r

# Step 2: Define tools for PL-Resolution

In [6]:
def resolve(a, b):
    # Init
    len_total = len(a) + len(b)
    r = a + b
    
    # 1: apply rule once
    for each in r:
        if neg(each) in r:
            r.remove(each)
            r.remove(neg(each))
            break
    
    # 2: check for tautology
    for each in r:
        if neg(each) in r:
            return None
        
    # 3.1: nothing changed
    if len(r) == len_total:
        return None
        
    # 3.2: something has changed
    # Remove duplicate literals
    r = set(r)
    # Sort literals alphabetically
    r = sorted(r, key=lambda c: c.replace('-', '') if '-' in c else c)
    # Repackage as a clause
    r = list(r)
    return r

In [7]:
def iterate(input_kb, verbose=False):
    kb       = deepcopy(input_kb)
    kb_check = deepcopy(input_kb)
    yields   = []

    for each_a in kb:
        kb_no_each = deepcopy(kb)
        kb_no_each.remove(each_a)
        for each_b in kb_no_each:
            # Apply resolution rule
            r = resolve(each_a, each_b)

            # Perform checks
            isNone    = r is None
            isInKB    = None
            isInYield = None
            if (not isNone):
                isInKB    = (r in kb_check or list(reversed(r)) in kb_check)
                isInYield = (r in yields   or list(reversed(r)) in yields)
            
            # Verbose log
            if verbose:
                status = '-'
                if isInKB: status = 'k'
                if isInYield: status = 'y'
                print('{s} {a} + {b} -> {r}'.format(
                    s=status, a=each_a, b=each_b, r=r
                ))
            
            # Check before adding to yield
            if (not isNone) and (not isInKB) and (not isInYield):
                yields.extend([r])
    
    return yields

# Step 3: Acutally do the resolution

In [8]:
def run(input_kb, input_alpha, verbose=False, verbose_iterate=False, write_to_file=False):
    global filename
    
    # File handler
    if write_to_file:
        f = open(output_file(filename), mode='w+', newline='\n')
    
    # Copy
    kb = deepcopy(input_kb)
    alpha = deepcopy(input_alpha)
    
    # Add not alpha to KB
    kb.extend(neg_alpha(alpha))
    
    # Iteration counter
    it = 0
    
    if verbose:
        print('Initial KB:')
        for i, each in enumerate(kb): print('{}: {}'.format(i, each))
        print()

    while True:
        if verbose:
            print('Iteration', it)

        y = iterate(kb, verbose_iterate)
        if verbose:
            if len(y) > 0:
                print('yield length =', len(y))
                for i, each in enumerate(y): print('{}: {}'.format(i, each))
            else:
                print('empty yield')
        
        # Write to file: yields
        if write_to_file:
            f.write('{}'.format(len(y)))
            f.write('\n')
            for each in y:
                if each == []:
                    f.write('{}')
                    f.write('\n')
                else:
                    f.write(' OR '.join(each))
                    f.write('\n')

        # Reverse check
        kb.extend(y)
        if [] in kb:
            if verbose:
                print('YES')
            if write_to_file:
                f.write('YES')
                f.close()
            break

        if len(y) == 0:
            if verbose:
                print('NO')
            if write_to_file:
                f.write('NO')
                f.close()
            break

        # Readability fix
        if verbose:
            print()
        
        it += 1

---

# Step 4: Profit?

In [9]:
alpha, kb = None, None
filename = 'Sample_alpha.txt'
with open(input_file(filename), 'r') as f:
    alpha, kb = parse(f)
run(kb, alpha, verbose=True, verbose_iterate=True, write_to_file=False)

Initial KB:
0: ['-A', 'B']
1: ['B', '-C']
2: ['A', '-B', 'C']
3: ['-B']
4: ['-A']

Iteration 0
- ['-A', 'B'] + ['B', '-C'] -> None
- ['-A', 'B'] + ['A', '-B', 'C'] -> None
k ['-A', 'B'] + ['-B'] -> ['-A']
- ['-A', 'B'] + ['-A'] -> None
- ['B', '-C'] + ['-A', 'B'] -> None
- ['B', '-C'] + ['A', '-B', 'C'] -> None
- ['B', '-C'] + ['-B'] -> ['-C']
- ['B', '-C'] + ['-A'] -> None
- ['A', '-B', 'C'] + ['-A', 'B'] -> None
- ['A', '-B', 'C'] + ['B', '-C'] -> None
- ['A', '-B', 'C'] + ['-B'] -> None
- ['A', '-B', 'C'] + ['-A'] -> ['-B', 'C']
k ['-B'] + ['-A', 'B'] -> ['-A']
y ['-B'] + ['B', '-C'] -> ['-C']
- ['-B'] + ['A', '-B', 'C'] -> None
- ['-B'] + ['-A'] -> None
- ['-A'] + ['-A', 'B'] -> None
- ['-A'] + ['B', '-C'] -> None
y ['-A'] + ['A', '-B', 'C'] -> ['-B', 'C']
- ['-A'] + ['-B'] -> None
yield length = 2
0: ['-C']
1: ['-B', 'C']

Iteration 1
- ['-A', 'B'] + ['B', '-C'] -> None
- ['-A', 'B'] + ['A', '-B', 'C'] -> None
k ['-A', 'B'] + ['-B'] -> ['-A']
- ['-A', 'B'] + ['-A'] -> None
- ['-A'

In [10]:
# # TODO: export to PLRes.py
# # Init
# alpha, kb, filename = None, None, None

# # Get a filelist
# filelist = tuple(*os.walk(INPUT_DIR))[2]

# # For each file in the filelist...
# for each_file in filelist:
#     # Expose the filename globally - because run() does not have the filename as args
#     # last minute code .-.
#     filename = each_file
#     # Open as parse the file
#     with open(input_file(filename), 'r') as f:
#         alpha, kb = parse(f)
#     # Process
#     run(kb, alpha, verbose=False, verbose_iterate=False, write_to_file=True)
#     print('Done:', filename)

---

# (Dead) Experimental implementation

This was an attempt to implement generic components of PL (literals, clauses and connectives) to abstract away all the underlying details and enable writing higher-level code that actually evaluates the expressions in a symbolic way.

Suffice to say, it did not go well. At least not with the time budget alloted to this Lab.

In [11]:
# class OrClause:
#     def __init__(self, init):
#         self.literals = []
#         if isinstance(init, str):
#             self.__parse(init)
#         elif isinstance(init, list):
#             self.literals = init
    
#     def __str__(self):
#         return ' \/ '.join(self.literals)
    
#     def __parse(self, clause_str):
#         # Split and strip the original string into literals
#         # (negation is kept as-is)
#         self.literals.extend(
#             c.strip() for c in clause_str.split(' OR ')
#         )
#         # Sort the literals, regardless of negation
#         self.literals = sorted(
#             self.literals,
#             key=lambda c:
#                 c.replace('-', '') if '-' in c
#                 else c
#         )
    
#     def __and__(self, another_clause):
#         # Attempt to AND with another OrClause
#         if not isinstance(another_clause, OrClause):
#             pass
#         r = [
#             OrClause(list(c))
#             for c in product(self.literals, another_clause.literals)        ]
#         return r

# class AndClause:
#     class OrClause:
#     def __init__(self, init):
#         self.literals = []
#         if isinstance(init, str):
#             self.__parse(init)
#         elif isinstance(init, list):
#             self.literals = init
    
#     def __str__(self):
#         return ' /\ '.join(self.literals)
    
#     def __parse(self, clause_str):
#         # Split and strip the original string into literals
#         # (negation is kept as-is)
#         self.literals.extend(
#             c.strip() for c in clause_str.split(' AND ')
#         )
#         # Sort the literals, regardless of negation
#         self.literals = sorted(
#             self.literals,
#             key=lambda c:
#                 c.replace('-', '') if '-' in c
#                 else c
#         )
    
#     def __or__(self, another_clause):
#         # Attempt to OR with another AndClause
#         if not isinstance(another_clause, AndClause):
#             pass
#         r = [
#             OrClause(list(c))
#             for c in product(self.literals, another_clause.literals)        ]
#         return r

In [12]:
# s1 = Clause('-A OR -B')
# s2 = Clause('A OR C')
# print(s1)
# print(s2)
# for each in s1 & s2:
#     print(each)